In [1]:
import funcx
import sys
import psij

print("Python version : ", sys.version)
print("FuncX version : ", funcx.__version__)
print("Psij version: ", psij.__version__)

Python version :  3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 15:55:03) 
[GCC 10.4.0]
FuncX version :  1.0.10
Psij version:  0.1.0.post2


In [2]:
theta_ep = "21d84df5-7eed-4b95-a4de-06b9c2b5a3d0"
midway_ep = "08e00632-4337-443d-822f-0948992bf0d3"
bebop_ep = "50906a25-f987-480c-a0dd-6e187c0d0f23"

In [3]:
fx_theta = funcx.FuncXExecutor(endpoint_id=theta_ep)
fx_midway = funcx.FuncXExecutor(endpoint_id=midway_ep)
fx_bebop = funcx.FuncXExecutor(endpoint_id=bebop_ep)

Please authenticate with Globus here:
------------------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall+openid+urn%3Aglobus%3Aauth%3Ascope%3Asearch.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=ytjm5eTMzSLtlrpaLjKNl9_H26CjBP3GKIenNO5szwo&code_challenge_method=S256&access_type=offline&prefill_named_grant=jupyter-emews-2dpsij-2dfuncx-2dexample-2d6amhfokr&prompt=login
------------------------------------



Enter the resulting Authorization Code here:  8zkMncna6FYmh7XKQ1Qbxy7CENXROe


In [4]:
def platinfo():
    import platform
    return platform.uname()

In [ ]:
fu_theta = fx_theta.submit(platinfo)

In [ ]:
fu_theta.result()

In [ ]:
fu_midway = fx_midway.submit(platinfo)

In [ ]:
fu_midway.result()

In [5]:
fu_bebop = fx_bebop.submit(platinfo)

In [6]:
fu_bebop.result()

uname_result(system='Linux', node='beboplogin1', release='3.10.0-1160.62.1.el7.x86_64', version='#1 SMP Tue Apr 5 16:57:59 UTC 2022', machine='x86_64')

In [ ]:
def submit_job():
    import logging
    logging.basicConfig(filename='', encoding='utf-8', level=logging.DEBUG)
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    
    executor = JobExecutor.get_instance("slurm")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                queue_name="broadwl",
                project_name="pi-chard",
            ),
        )
    )

    executor.submit(job)
    return job.native_id

In [ ]:
job_fu = fx.submit(submit_job, endpoint_id=midway_ep)

In [ ]:
print(job_fu.result())
status_id = job_fu.result()

In [27]:
def status_job(native_id):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance("slurm")
    job = Job()
    executor.attach(job, native_id)
    time.sleep(5)
    return job.status

In [ ]:
status_fu = fx.submit(status_job, status_id, endpoint_id=midway_ep)

In [ ]:
x = status_fu.result()

In [ ]:
str(x)

In [ ]:
def submit_job_cobalt():
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    from psij import JobExecutorConfig
    
    executor = JobExecutor.get_instance("cobalt")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                project_name='CSC249ADTR01',
                queue_name='debug-cache-quad'
            ),
        )
    )

    executor.submit(job)
    
    return job.native_id

In [ ]:
job_fu = fx_theta.submit(submit_job_cobalt)

In [ ]:
job_id = job_fu.result()
print("Job_id : ", job_id)

In [ ]:
def status_job(native_id):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance("cobalt")
    job = Job()
    executor.attach(job, native_id)
    time.sleep(3)
    return job.status

In [ ]:
status_fu = fx_theta.submit(status_job, job_id)

s = status_fu.result()

In [ ]:
print(s)

In [30]:
def cancel_job(native_id, scheduler="cobalt"):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance(scheduler)
    job = Job()
    executor.attach(job, native_id)
    time.sleep(3)
    return job.cancel()

In [ ]:
cancel_fu = fx_theta.submit(cancel_job, job_id)

s = cancel_fu.result()
print(s)

In [8]:
def submit_job_bebop():
    import logging
    logging.basicConfig(filename='', encoding='utf-8', level=logging.DEBUG)
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    
    executor = JobExecutor.get_instance("slurm")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                queue_name="bdws",
                project_name="EMEWS",
            ),
        )
    )

    executor.submit(job)
    return job.native_id

In [31]:
future = fx_bebop.submit(submit_job_bebop)

In [32]:
job_id = future.result()
print(f"Job id on bebop: {job_id}")

Job id on bebop: 2719218


In [33]:
future = fx_bebop.submit(status_job, job_id)

In [34]:
status = future.result()
print(status)

JobStatus[ACTIVE, time=1675876943.47338]


In [35]:
future = fx_bebop.submit(cancel_job, job_id, scheduler="slurm")

In [36]:
print("Output from cancel:", future.result())

Output from cancel: None
